# 🛠️ Part 1: Setup & Dependencies

First, we import the core components.
*   **ChatOllama**: The interface to our local Llama 3 model.
*   **Prompts**: Templates to structure how we talk to the AI.
*   **History**: Tools to save conversation data into a SQL database so the bot remembers us even if we restart the script.


In [1]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)

# For persistent memory
from langchain_core.messages import HumanMessage
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import SQLChatMessageHistory

print("✅ Imports loaded successfully.")


✅ Imports loaded successfully.


# 🤖 Part 2: Initialize the LLM

We configure the connection to Ollama.
*   **temperature=0.8**: Set to be slightly creative.
*   **num_predict=256**: Limits the response length to keep it concise.

*(Note: 'temerature' was a typo in the original code, corrected to 'temperature' here).*


In [2]:
base_url = "http://localhost:11434"
model = 'llama3.2:latest'

llm = ChatOllama(
    base_url=base_url,
    model=model,
    temperature=0.8,
    num_predict=256,
)

print(f"✅ Model {model} connected.")


✅ Model llama3.2:latest connected.


# ⚠️ Part 3: The Problem (Stateless Chains)

First, let's look at what happens if we **don't** use memory components.
Standard LLM chains are "Stateless". They treat every input as a brand new conversation.

Below, we tell the AI our name, but in the very next line, it will likely forget because we didn't pass the history back to it.


In [4]:
# 1. Simple Template
simple_template = ChatPromptTemplate.from_template("{prompt}")
simple_chain = simple_template | llm | StrOutputParser()

# 2. We introduce ourselves
print("👤 User: My name is Arash...")
print(f"🤖 AI: {simple_chain.invoke({'prompt': 'My name is Arash, I work as AI Consultant.'})}")

# 3. We ask for recall
print("\n👤 User: What is my name?")
response = simple_chain.invoke({'prompt': 'what is my name?'})
print(f"🤖 AI: {response}")

print("\n❌ Notice: The AI likely forgot the name because this chain has no memory.")


👤 User: My name is Arash...
🤖 AI: Hello Arash! It's nice to meet you. As an AI Consultant, I'm sure you have a fascinating career helping organizations leverage the power of artificial intelligence.

What aspects of AI are you most interested in or passionate about? Are you working with natural language processing, computer vision, machine learning, or perhaps something else?

Feel free to share more about your work and interests. I'm here to listen and help if I can!

👤 User: What is my name?
🤖 AI: I don't have any information about your name. I'm a large language model, I don't retain personal data or have the ability to know individual users' names unless you tell me. If you'd like to share your name with me, I'd be happy to chat with you!

❌ Notice: The AI likely forgot the name because this chain has no memory.


# 🧠 Part 4: The Architecture for Memory

To fix the problem above, we need a **Prompt Structure** that reserves a specific "slot" for history.

1.  **System Message**: Defines the AI's personality.
2.  **MessagesPlaceholder("history")**: This is the magic part. It tells LangChain: *"Insert all past messages right here before the new user input."*
3.  **Human Message**: The current question.


In [5]:
# Define the components
system_msg = SystemMessagePromptTemplate.from_template("You are a helpful assistant.")
human_msg = HumanMessagePromptTemplate.from_template("{input}")

# Create the structure with a placeholder for 'history'
messages = [
    system_msg,
    MessagesPlaceholder(variable_name="history"), # <--- The Memory Slot
    human_msg
]

# Build the template and the chain
memory_prompt = ChatPromptTemplate(messages=messages)
memory_chain = memory_prompt | llm | StrOutputParser()

print("✅ Memory-aware chain created.")


✅ Memory-aware chain created.


# 💾 Part 5: Wiring the Memory (SQL Database)

Now we connect our chain to a SQLite database.
*   **SQLChatMessageHistory**: Automatically saves inputs and outputs to a local file (`chat_history.db`).
*   **RunnableWithMessageHistory**: This is the manager. It takes our `memory_chain`, looks at the `history` slot we defined, and automatically fills it with data from the database.

**Key Parameters:**
*   `input_messages_key='input'`: Matches our prompt's `{input}`.
*   `history_messages_key='history'`: Matches our `MessagesPlaceholder`.


In [6]:
def get_session_history(session_id):
    """
    Creates a connection to a SQLite DB for a specific session ID.
    If the file doesn't exist, it creates it.
    """
    connection = "sqlite:///chat_history.db"
    return SQLChatMessageHistory(session_id, connection)

# Wrap the chain with the history manager
runnable_with_history = RunnableWithMessageHistory(
    memory_chain,
    get_session_history,
    input_messages_key='input',
    history_messages_key='history'
)

print("✅ Runnable wired with SQL Database.")


✅ Runnable wired with SQL Database.


# 🚀 Part 6: Testing the Persistent Chatbot

Now we define a clean function `chat_with_llm` that takes a `session_id` (like a User ID).
We will test it by:
1.  Teaching it a fact.
2.  Asking it to recall the fact.

*Note: Since this saves to a database, if you restart this notebook and use the same 'arash' ID, it will still remember you!*


In [7]:
def chat_with_llm(session_id, user_input):
    print(f"👤 User ({session_id}): {user_input}")
    
    output = runnable_with_history.invoke(
        {'input': user_input},
        config={"configurable": {"session_id": session_id}}
    )
    
    print(f"🤖 AI: {output}\n")
    return output

# --- Test Sequence ---

user_id = 'arash_v1'

# 1. Provide Context
chat_with_llm(user_id, "My name is Arash, I work as AI Consultant.")

# 2. Ask for Recall (This should work now!)
chat_with_llm(user_id, "What is my job?")


👤 User (arash_v1): My name is Arash, I work as AI Consultant.


c:\Users\arash\AppData\Local\ooba\ooba\text-generation-ui\installer_files\conda\lib\site-packages\langchain_core\runnables\history.py:600: LangChainDeprecationWarning: `connection_string` was deprecated in LangChain 0.2.2 and will be removed in 1.0. Use connection instead.
  message_history = self.get_session_history(


🤖 AI: Hello Arash! It's nice to meet you. As an AI Consultant, I'm sure you have a deep understanding of the latest advancements and applications in artificial intelligence.

What can I help you with today? Do you have any questions or topics you'd like to discuss regarding AI? Or perhaps you need assistance with a project or problem you're working on?

👤 User (arash_v1): What is my job?
🤖 AI: Your job, Arash, is as an AI Consultant. This means you help organizations and businesses implement and integrate artificial intelligence solutions into their operations. Your role involves assessing client needs, recommending suitable AI technologies and strategies, and guiding clients through the process of implementing these solutions.

As an AI Consultant, your expertise likely spans a range of areas, including natural language processing, machine learning, computer vision, and more. You help clients to identify opportunities for automation, improve efficiency, enhance customer experiences, a

'Your job, Arash, is as an AI Consultant. This means you help organizations and businesses implement and integrate artificial intelligence solutions into their operations. Your role involves assessing client needs, recommending suitable AI technologies and strategies, and guiding clients through the process of implementing these solutions.\n\nAs an AI Consultant, your expertise likely spans a range of areas, including natural language processing, machine learning, computer vision, and more. You help clients to identify opportunities for automation, improve efficiency, enhance customer experiences, and gain a competitive edge in their respective industries.'